<a href="https://colab.research.google.com/github/yanann11/nebius_llm_course/blob/main/BrokenTelephone_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: get longest word from the sentance

def longest_word(sentence):
    longest = ""
    for word in sentence.split():
        if len(word) > len(longest):
            longest = word
    return longest
sentence = "This is a sample sentence"
print(longest_word(sentence))


sentence


In [5]:
print(longest_word("Hello wold, let's write something amazing!"))
from google.colab import files
uploaded = files.upload()

something


Saving ngrock_authtoken to ngrock_authtoken (1)


It is my test notebook!

In [1]:
!pip uninstall -y flask flask-ngrok jinja2 markupsafe werkzeug
!pip install werkzeug==2.0.3
!pip install markupsafe==2.0.1
!pip install jinja2==2.11.3
!pip install flask==2.0.1
!pip install flask-ngrok
!pip install pyngrok

Found existing installation: Flask 3.1.1
Uninstalling Flask-3.1.1:
  Successfully uninstalled Flask-3.1.1
Found existing installation: Jinja2 3.1.6
Uninstalling Jinja2-3.1.6:
  Successfully uninstalled Jinja2-3.1.6
Found existing installation: MarkupSafe 3.0.2
Uninstalling MarkupSafe-3.0.2:
  Successfully uninstalled MarkupSafe-3.0.2
Found existing installation: Werkzeug 3.1.3
Uninstalling Werkzeug-3.1.3:
  Successfully uninstalled Werkzeug-3.1.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.2/289.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidi

In [8]:
import os

with open("ngrok_authtoken", "r") as file:
    ngrok_authtoken = file.read().strip()

os.environ["NGROK_AUTHTOKEN"] = ngrok_authtoken

In [10]:
!pip install flask pyngrok

In [11]:
# === SERVER (run this in the first Colab notebook) ===

from flask import Flask, request, jsonify
import base64
from io import BytesIO
from PIL import Image
from pyngrok import ngrok

app = Flask(__name__)

# Initialize the game
game = None

def init_game(client):
    """Initialize the game with Nebius client"""
    global game
    game = BrokenTelephone(client=client)

@app.route('/play', methods=['POST'])
def play_game():
    try:
        data = request.json
        n_rounds = data.get('n_rounds', 3)
        starting_prompt = data.get('starting_prompt')

        if not starting_prompt:
            return jsonify({'error': 'starting_prompt is required'}), 400

        # Play the game
        history = game.play_game(n_rounds=n_rounds, starting_prompt=starting_prompt)

        # Convert history to serializable format
        serialized_history = []
        for description, image in history:
            # Convert PIL Image to base64
            buffered = BytesIO()
            image.save(buffered, format="PNG")
            img_str = base64.b64encode(buffered.getvalue()).decode()

            serialized_history.append({
                'description': description,
                'image': img_str
            })

        return jsonify({
            'status': 'success',
            'history': serialized_history
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

def start_server(client):
    """Start the server with Nebius client"""
    try:
        # Initialize the game
        init_game(client)

        # Start ngrok
        public_url = ngrok.connect(5000)
        print(f"\nNgrok tunnel established at: {public_url}")
        print("Use this URL in your client notebook")

        # Start Flask app
        app.run(port=5000)
    except Exception as e:
        print(f"Error starting server: {str(e)}")

In [ ]:
# Initialize the client
client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY")
)

# Start the server
start_server(client)